In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import networkx as nx
import matplotlib.pyplot as plt
import random

# Create a class for the Genetic Algorithm to solve graph coloring
class GeneticAlgorithm:
    def __init__(self, graph, population_size, mutation_rate, generations):
        self.graph = graph
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = []
        self.initialize_population()

    def initialize_population(self):
        num_nodes = len(self.graph.nodes)
        for _ in range(self.population_size):
            # Randomly initialize a color assignment
            individual = [random.randint(0, num_nodes - 1) for _ in range(num_nodes)]
            self.population.append(individual)

    def fitness(self, individual):
        # Fitness is measured by the number of conflicting edges
        conflicts = 0
        for edge in self.graph.edges:
            if individual[edge[0]] == individual[edge[1]]:
                conflicts += 1
        return -conflicts  # Negative conflicts, as lower is better

    def select_parents(self):
        # Tournament selection
        selected = random.sample(self.population, 2)
        return max(selected, key=self.fitness)

    def crossover(self, parent1, parent2):
        # Single-point crossover
        crossover_point = random.randint(1, len(parent1) - 1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        return child

    def mutate(self, individual):
        # Random mutation
        if random.random() < self.mutation_rate:
            mutate_point = random.randint(0, len(individual) - 1)
            individual[mutate_point] = random.randint(0, len(self.graph.nodes) - 1)

    def evolve(self):
        new_population = []
        for _ in range(self.population_size):
            parent1 = self.select_parents()
            parent2 = self.select_parents()
            child = self.crossover(parent1, parent2)
            self.mutate(child)
            new_population.append(child)
        self.population = new_population

    def run(self):
        best_individual = None
        best_fitness = float('-inf')
        for _ in range(self.generations):
            self.evolve()
            for individual in self.population:
                fitness = self.fitness(individual)
                if fitness > best_fitness:
                    best_fitness = fitness
                    best_individual = individual
        return best_individual

# Create the GUI Application
class GraphColoringApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Graph Coloring with Genetic Algorithm")
        self.geometry("400x300")
        self.graph = nx.Graph()
        self.population_size = 10
        self.mutation_rate = 0.1
        self.generations = 50

        # GUI Components
        self.create_widgets()

    def create_widgets(self):
        self.load_graph_button = tk.Button(self, text="Load Graph", command=self.load_graph)
        self.load_graph_button.pack(pady=10)

        self.population_label = tk.Label(self, text="Population Size:")
        self.population_label.pack()
        self.population_entry = tk.Entry(self)
        self.population_entry.insert(0, str(self.population_size))
        self.population_entry.pack()

        self.mutation_label = tk.Label(self, text="Mutation Rate:")
        self.mutation_label.pack()
        self.mutation_entry = tk.Entry(self)
        self.mutation_entry.insert(0, str(self.mutation_rate))
        self.mutation_entry.pack()

        self.generations_label = tk.Label(self, text="Generations:")
        self.generations_label.pack()
        self.generations_entry = tk.Entry(self)
        self.generations_entry.insert(0, str(self.generations))
        self.generations_entry.pack()

        self.run_button = tk.Button(self, text="Run Genetic Algorithm", command=self.run_genetic_algorithm)
        self.run_button.pack(pady=10)

        self.show_graph_button = tk.Button(self, text="Show Graph", command=self.show_graph)
        self.show_graph_button.pack(pady=10)

    def load_graph(self):
        filename = filedialog.askopenfilename(title="Select Graph File")
        if not filename:
            return

        # Clear existing graph
        self.graph.clear()

        try:
            self.graph = nx.read_edgelist(filename)
            messagebox.showinfo("Graph Loaded", "Graph has been successfully loaded.")
        except Exception as e:
            messagebox.showerror("Error", f"Could not load graph: {str(e)}")

    def run_genetic_algorithm(self):
        try:
            self.population_size = int(self.population_entry.get())
            self.mutation_rate = float(self.mutation_entry.get())
            self.generations = int(self.generations_entry.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Please ensure that population size, mutation rate, and generations are correctly entered.")
            return

        if self.graph.number_of_nodes() == 0:
            messagebox.showerror("No Graph", "Please load a graph before running the genetic algorithm.")
            return

        ga = GeneticAlgorithm(self.graph, self.population_size, self.mutation_rate, self.generations)
        best_solution = ga.run()

        # Display the solution
        result_text = "\n".join([f"Node {i}: Color {best_solution[i]}" for i in range(len(best_solution))])
        messagebox.showinfo("Genetic Algorithm Result", f"Best Solution:\n{result_text}")

    def show_graph(self):
        if self.graph.number_of_nodes() == 0:
            messagebox.showerror("No Graph", "Please load a graph before showing it.")
            return

        plt.figure()
        pos = nx.spring_layout(self.graph)
        nx.draw(self.graph, pos, with_labels=True, node_color="lightblue", edge_color="gray")
        plt.show()

# Run the Application
if __name__ == "__main__":
    app = GraphColoringApp()
    app.mainloop()
